In [1]:
# Inelegant but we are only loading three files:
topics_35 = open('../output/nmf_35_topics.txt', 'r').read().splitlines()
topics_45 = open('../output/nmf_45_topics.txt', 'r').read().splitlines()
topics_55 = open('../output/nmf_55_topics.txt', 'r').read().splitlines()

In [2]:
# The Jaccard coefficient is going to be our measure of similarity. 
# (This is entirely provisional, but it performs better here than the
# edit distance.)
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [4]:
for ik, iv in enumerate(topics_45):
    for jk, jv in enumerate(topics_35):
        if jaccard_similarity(iv.split(" "), jv.split(" ")) > 0.35:
            print(f"35-{ik} and 45-{jk} match.")

35-0 and 45-0 match.
35-1 and 45-1 match.
35-2 and 45-2 match.
35-3 and 45-3 match.
35-4 and 45-4 match.
35-5 and 45-5 match.
35-7 and 45-7 match.
35-8 and 45-8 match.
35-9 and 45-9 match.
35-10 and 45-10 match.
35-11 and 45-11 match.
35-12 and 45-12 match.
35-13 and 45-13 match.
35-14 and 45-14 match.
35-15 and 45-15 match.
35-16 and 45-16 match.
35-17 and 45-17 match.
35-18 and 45-18 match.
35-19 and 45-22 match.
35-20 and 45-20 match.
35-21 and 45-21 match.
35-23 and 45-23 match.
35-25 and 45-25 match.
35-27 and 45-31 match.
35-28 and 45-28 match.
35-29 and 45-29 match.
35-30 and 45-33 match.
35-32 and 45-24 match.
35-35 and 45-34 match.
35-36 and 45-30 match.
35-37 and 45-6 match.
35-38 and 45-27 match.
35-40 and 45-32 match.
35-42 and 45-15 match.
35-44 and 45-26 match.
